In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
from torchvision.transforms import v2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(SimpleCNN, self).__init__()

        self.n_channels = n_channels
        self.n_classes = n_classes

        self.conv1 = nn.Conv2d(in_channels=n_channels, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.sigmoid(x)
        x = self.pool2(x)
        x = self.conv2(x)
        x = torch.sigmoid(x)
        x = self.pool2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
num_epochs = 3
durations = []

print("=== GPU Computing Time ===")
model = SimpleCNN(n_channels=1, n_classes=10).to(device)
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(num_epochs):
    start_time = time.time()  # Record start time
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    end_time = time.time()  # Record start time
    epoch_duration = end_time - start_time  # Calculate elapsed time
    durations.append(epoch_duration)
    print(f"Epoch {epoch+1}/{num_epochs} - Time: {epoch_duration:.2f} seconds")
gpu_avg_duration = np.mean(durations)
print(f"Average duration:  {gpu_avg_duration:.2f} seconds")

=== GPU Computing Time ===
Epoch 1/3 - Time: 14.28 seconds
Epoch 2/3 - Time: 12.68 seconds
Epoch 3/3 - Time: 12.71 seconds
Average duration:  13.22 seconds
